In [1]:
import astropy as ast
from astropy.coordinates import solar_system_ephemeris, EarthLocation, GeocentricTrueEcliptic, get_body, SkyCoord, Distance
from astroplan.moon import moon_phase_angle
from collections import defaultdict
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

### Generating Data From Celestial Bodies

In [2]:
# All of the celestial bodies to generate synthetic data on
BODY_NAMES = ['mercury', 'venus', 'mars', 'jupiter', 'saturn', 'moon', 'sun']

In [3]:
def get_coordinates(body):
    # Takes a Skycoord object, returns (theta, phi, r, x, y, z) in (deg, deg, AU, AU, AU, AU)
    angles = [float(i) for i in body.to_string().split(' ')]
    body_dist_string = body.distance.to_string()
    r = float(body_dist_string[:-3])
    units = body_dist_string[-2:]
    if units == 'km':
        r /= 1.496e+8
    
    phi = angles[0]
    theta = angles[1]

    # Extract the Cartesian coordinates from the SkyCoord object
    c = body.cartesian
    x = c.x.value
    y = c.y.value
    z = c.z.value

    body_dist_string = body.distance.to_string()
    units = body_dist_string[-2:]
    
    # Convert from km to AU if necessary
    if units == 'km':
        x /= 1.496e+8
        y /= 1.496e+8
        z /= 1.496e+8
    return (phi, theta, r, x, y, z)

In [11]:
def add_noise(coords):
    # Takes a tuple of (theta, phi, r, x, y, z) in (deg, deg, AU, AU, AU, AU)
    # and returns a noisified version of the data
    # Currently doesn't add noise to r
    theta, phi, r, x, y, z = coords
    return (theta + np.random.normal(0,1), phi + np.random.normal(0,1), r + np.random.normal(0, 0.1),
            x + np.random.normal(0, 0.1), y + np.random.normal(0, 0.1), z + np.random.normal(0, 0.1))

# Range is 150 years, almost the limit available to us from the astropy API
times = pd.date_range(start="1850-01-01-00-00-00", end="2000-01-01-00-00-00", freq='5D')

# Location is the Medicina Radio Observatory, located in Italy. Chosen for proximity to Greece
loc = EarthLocation.of_site('medicina')
rows = defaultdict(list)


In [12]:
# Generate coordinate data in terms of spherical Geocentric Celestial Reference System (GCRS), default for astropy
for time in tqdm_notebook(times):
    time = ast.time.Time(time.to_pydatetime())
    bodies = []
    
    with solar_system_ephemeris.set('builtin'):
        for body_name in BODY_NAMES:
            bodies.append(get_body(body_name, time, loc))

    rows['time'].append(time)
    rows['location'].append(str(loc))
    rows['moon_phase'].append(moon_phase_angle(time).value)

    for body_name, body in zip(BODY_NAMES, bodies):
        coordinates = add_noise(get_coordinates(body))
        coord_strings = ['theta', 'phi', 'r', 'x', 'y', 'z']
        for i in range(len(coord_strings)):
            c = coordinates[i]
            rows[body_name + '_' + coord_strings[i]].append(c)

celestial_bodies = pd.DataFrame(rows)
celestial_bodies.to_csv('data.csv', index=False)

/Users/kaahan/miniconda3/envs/project_s/lib/python3.6/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


KeyboardInterrupt: 

In [13]:
# Load the generated data (which needs to be converted to the geocentric ecliptic coordinate system)
data = pd.read_csv('data.csv')

In [18]:
# Convert coordinates to the standard geocentric true ecliptic coordinate system
# See https://docs.astropy.org/en/stable/api/astropy.coordinates.GeocentricTrueEcliptic.html for more documentation

rows = defaultdict(list)
for name in BODY_NAMES:
    phi_col = data[name + '_phi']
    theta_col = data[name + '_theta']
    r_col = data[name + '_r']
    for phi, theta, r in tqdm_notebook(zip(phi_col, theta_col, r_col)):
        ecliptic = SkyCoord(theta, phi, abs(r), frame='gcrs', unit=('deg', 'deg', 'AU')).transform_to(GeocentricTrueEcliptic())
        print(phi, theta, r)
        print(abs(r))
        print(ecliptic)
        coordinates = get_coordinates(ecliptic)
        coord_strings = ['lambda', 'beta', 'delta', 'x', 'y', 'z']
        for i in range(len(coord_strings)):
            c = coordinates[i]
            rows[name + '_' + coord_strings[i]].append(c)

/Users/kaahan/miniconda3/envs/project_s/lib/python3.6/site-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


-25.27693818 290.6611833 1.399639745
1.399639745
<SkyCoord (GeocentricTrueEcliptic: equinox=J2000.000, obstime=J2000.000): (lon, lat, distance) in (deg, deg, AU)
    (288.63156117, -3.16444327, 1.39963974)>
-21.99803223 300.3610697 1.3018584979999999
1.3018584979999999
<SkyCoord (GeocentricTrueEcliptic: equinox=J2000.000, obstime=J2000.000): (lon, lat, distance) in (deg, deg, AU)
    (297.95262356, -1.45711939, 1.3018585)>
-21.11781753 311.0837494 1.2621107790000001
1.2621107790000001
<SkyCoord (GeocentricTrueEcliptic: equinox=J2000.000, obstime=J2000.000): (lon, lat, distance) in (deg, deg, AU)
    (307.86252102, -2.91579569, 1.26211078)>
-16.33248547 317.81745939999996 1.063264707
1.063264707
<SkyCoord (GeocentricTrueEcliptic: equinox=J2000.000, obstime=J2000.000): (lon, lat, distance) in (deg, deg, AU)
    (315.32123383, -0.09623246, 1.06326471)>
-14.69392536 324.0639314 0.8899528040000001
0.8899528040000001
<SkyCoord (GeocentricTrueEcliptic: equinox=J2000.000, obstime=J2000.000): (

    (186.42969939, -3.19385507, 1.31946108)>
-7.638705185 191.2062974 1.053169317
1.053169317
<SkyCoord (GeocentricTrueEcliptic: equinox=J2000.000, obstime=J2000.000): (lon, lat, distance) in (deg, deg, AU)
    (193.28345468, -2.59858137, 1.05316932)>
-9.795270835 196.40329440000002 1.079051554
1.079051554
<SkyCoord (GeocentricTrueEcliptic: equinox=J2000.000, obstime=J2000.000): (lon, lat, distance) in (deg, deg, AU)
    (198.8601469, -2.60190147, 1.07905155)>
-13.6084769 198.6255531 0.7552945879999999
0.7552945879999999
<SkyCoord (GeocentricTrueEcliptic: equinox=J2000.000, obstime=J2000.000): (lon, lat, distance) in (deg, deg, AU)
    (202.33091162, -5.30136438, 0.75529459)>
-14.06656386 203.21600180000002 0.8914503109999999
0.8914503109999999
<SkyCoord (GeocentricTrueEcliptic: equinox=J2000.000, obstime=J2000.000): (lon, lat, distance) in (deg, deg, AU)
    (206.65215583, -4.06520904, 0.89145031)>
-12.60789234 201.9947763 0.680630895
0.680630895
<SkyCoord (GeocentricTrueEcliptic: equ

    (65.12568365, 1.66945182, 0.65831827)>
20.59352736 60.38120623 0.485882022
0.485882022
<SkyCoord (GeocentricTrueEcliptic: equinox=J2000.000, obstime=J2000.000): (lon, lat, distance) in (deg, deg, AU)
    (62.43816301, -0.05685761, 0.48588202)>
17.427804300000002 57.94321504 0.462098492
0.462098492
<SkyCoord (GeocentricTrueEcliptic: equinox=J2000.000, obstime=J2000.000): (lon, lat, distance) in (deg, deg, AU)
    (59.53512042, -2.68585838, 0.46209849)>
16.65839752 56.19100701 0.468009713
0.468009713
<SkyCoord (GeocentricTrueEcliptic: equinox=J2000.000, obstime=J2000.000): (lon, lat, distance) in (deg, deg, AU)
    (57.73074559, -3.07427954, 0.46800971)>
14.692108 55.73940132 0.796770993
0.796770993
<SkyCoord (GeocentricTrueEcliptic: equinox=J2000.000, obstime=J2000.000): (lon, lat, distance) in (deg, deg, AU)
    (56.86595337, -4.89393314, 0.79677099)>
17.48276655 55.42292101 0.676294823
0.676294823
<SkyCoord (GeocentricTrueEcliptic: equinox=J2000.000, obstime=J2000.000): (lon, lat,

-25.37643502 264.3305542 1.3517376559999998
1.3517376559999998
<SkyCoord (GeocentricTrueEcliptic: equinox=J2000.000, obstime=J2000.000): (lon, lat, distance) in (deg, deg, AU)
    (264.87201184, -2.03794121, 1.35173766)>
-27.07916976 270.9213687 1.31048343
1.31048343
<SkyCoord (GeocentricTrueEcliptic: equinox=J2000.000, obstime=J2000.000): (lon, lat, distance) in (deg, deg, AU)
    (270.81815165, -3.64251754, 1.31048343)>
-24.06662118 280.039254 1.132115867
1.132115867
<SkyCoord (GeocentricTrueEcliptic: equinox=J2000.000, obstime=J2000.000): (lon, lat, distance) in (deg, deg, AU)
    (279.15607302, -0.9459989, 1.13211587)>
-24.542968 286.25935039999996 1.135955661
1.135955661
<SkyCoord (GeocentricTrueEcliptic: equinox=J2000.000, obstime=J2000.000): (lon, lat, distance) in (deg, deg, AU)
    (284.75988217, -1.93317139, 1.13595566)>



KeyboardInterrupt: 

In [ ]:
# Prepare the final dataset as final_data.csv

final_data = pd.DataFrame(rows)
final_data['time'] = data['time'] # Time is given in yyyy-mm-dd hh:mm:ss format
final_data['location'] = data['location'] # Location is given as (longitude, latitude, height) in m
final_data['moon_phase'] = data['moon_phase'] # Moon phase 
final_data.to_csv('final_data.csv', index=False) # All other columns are the spherical and Cartesian geocentric true ecliptic coordinate system values as described in the writeup

In [ ]:
final_data